In [1]:
# %pip install -qU langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [1]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class SubQuery(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    sub_query: str = Field(
        ...,
        description="A very specific query against the database.",
    )

In [10]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \

Perform query decomposition. Given a user question, break it down into distinct sub questions that \
you need to answer in order to answer the original question.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(model="qwen2.5:14b", temperature=0, base_url="http://localhost:11434/v1", api_key="sk-xxx")
llm_with_tools = llm.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | llm_with_tools | parser

In [11]:
query_analyzer.invoke({"question": "how to do rag"})

[SubQuery(sub_query='What is RAG?')]

In [12]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)

[SubQuery(sub_query='how to create a chain using multi-modal models'),
 SubQuery(sub_query='how to turn a chain into a rest api')]

In [13]:
examples = []
question = "What's chat langchain, is it a langchain template?"
queries = [
    SubQuery(sub_query="What is chat langchain"),
    SubQuery(sub_query="What is a langchain template"),
]
examples.append({"input": question, "tool_calls": queries})

In [14]:
question = "How would I use LangGraph to build an automaton"
queries = [
    SubQuery(sub_query="How to build automaton with LangGraph"),
]
examples.append({"input": question, "tool_calls": queries})

In [15]:
question = "How to build multi-agent system and stream intermediate steps from it"
queries = [
    SubQuery(sub_query="How to build multi-agent system"),
    SubQuery(sub_query="How to stream intermediate steps"),
    SubQuery(sub_query="How to stream intermediate steps from multi-agent system"),
]
examples.append({"input": question, "tool_calls": queries})

In [16]:
question = "What's the difference between LangChain agents and LangGraph?"
queries = [
    SubQuery(sub_query="What's the difference between LangChain agents and LangGraph?"),
    SubQuery(sub_query="What are LangChain agents"),
    SubQuery(sub_query="What is LangGraph"),
]
examples.append({"input": question, "tool_calls": queries})

In [17]:
question = "Is the teacher standing in front of the classroom while giving a lecture or conducting an activity?"
queries = [
    SubQuery(sub_query="Is the teacher standing in front of the classroom?"),
    SubQuery(sub_query="Is the teacher giving a lecture or conducting an activity?"),
]

examples.append({"input": question, "tool_calls": queries})

In [18]:
examples

[{'input': "What's chat langchain, is it a langchain template?",
  'tool_calls': [SubQuery(sub_query='What is chat langchain'),
   SubQuery(sub_query='What is a langchain template')]},
 {'input': 'How would I use LangGraph to build an automaton',
  'tool_calls': [SubQuery(sub_query='How to build automaton with LangGraph')]},
 {'input': 'How to build multi-agent system and stream intermediate steps from it',
  'tool_calls': [SubQuery(sub_query='How to build multi-agent system'),
   SubQuery(sub_query='How to stream intermediate steps'),
   SubQuery(sub_query='How to stream intermediate steps from multi-agent system')]},
 {'input': "What's the difference between LangChain agents and LangGraph?",
  'tool_calls': [SubQuery(sub_query="What's the difference between LangChain agents and LangGraph?"),
   SubQuery(sub_query='What are LangChain agents'),
   SubQuery(sub_query='What is LangGraph')]},
 {'input': 'Is the teacher standing in front of the classroom while giving a lecture or conductin

Update prompt templates

In [19]:
import uuid
from typing import Dict, List

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "This is an example of a correct usage of this tool. Make sure to continue using the tool this way."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages


example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]

In [20]:
example_msgs

[HumanMessage(content="What's chat langchain, is it a langchain template?"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2256441a-0ff7-4342-a15c-fcc0fc3cb7e8', 'type': 'function', 'function': {'name': 'SubQuery', 'arguments': '{"sub_query": "What is chat langchain"}'}}, {'id': 'c830070f-3226-4c24-814c-fe22d19e9b74', 'type': 'function', 'function': {'name': 'SubQuery', 'arguments': '{"sub_query": "What is a langchain template"}'}}]}, tool_calls=[{'name': 'SubQuery', 'args': {'sub_query': 'What is chat langchain'}, 'id': '2256441a-0ff7-4342-a15c-fcc0fc3cb7e8'}, {'name': 'SubQuery', 'args': {'sub_query': 'What is a langchain template'}, 'id': 'c830070f-3226-4c24-814c-fe22d19e9b74'}]),
 ToolMessage(content='This is an example of a correct usage of this tool. Make sure to continue using the tool this way.', tool_call_id='2256441a-0ff7-4342-a15c-fcc0fc3cb7e8'),
 ToolMessage(content='This is an example of a correct usage of this tool. Make sure to continue using the tool 

In [21]:
from langchain_core.prompts import MessagesPlaceholder

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \

Perform query decomposition. Given a user question, break it down into the most specific sub questions you can \
which will help you answer the original question. Each sub question should be about a single concept/fact/idea.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        ("human", "{question}"),
    ]
)
query_analyzer_with_examples = (
    prompt.partial(examples=example_msgs) | llm_with_tools | parser
)

In [22]:
query_analyzer_with_examples.invoke(
    {
        "question": "what's the difference between web voyager and reflection agents? do they use langgraph?"
    }
)

[SubQuery(sub_query='What is the difference between Web Voyager and Reflection Agents?'),
 SubQuery(sub_query='Do Web Voyager and Reflection Agents use LangGraph?'),
 SubQuery(sub_query='What are Web Voyager?'),
 SubQuery(sub_query='What are Reflection Agents?'),
 SubQuery(sub_query='Does Web Voyager use LangGraph?'),
 SubQuery(sub_query='Do Reflection Agents use LangGraph?')]

In [23]:
prompt

ChatPromptTemplate(input_variables=['question'], input_types={'examples': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'examples': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at converting user questions into database queries. You have access to a database of tutorial videos about a software library for building LLM-powered applications. \nPerform query decomposition. Given a user question, break it down into the most specific sub questions you can which will help you answer the original question. Each sub question should be about a single concept/fact/idea.\n\nIf there are acronyms or words you are not familiar with, do not try to rephrase them.')), Messages